<a href="https://colab.research.google.com/github/wangqianyang/colab/blob/main/my_torrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [1]:
# 使用3.11的版本
!python --version

Python 3.12.11


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtorrent
#!python -m pip install jedi
!python -m pip install --upgrade pip setuptools wheel && python -m pip install jedi libtorrent wget torf && apt install python3-libtorrent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.7 MB/s  0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
import libtorrent as lt
# 测试一下
print(f"成功导入 libtorrent 版本: {lt.version}")

成功导入 libtorrent 版本: 2.0.11.0


## 下载磁力链接

In [7]:
import libtorrent as lt
import time
import sys

# 创建一个 libtorrent 会话
ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

# 在这里输入您的磁力链接
#magnet_link = "magnet:?xt=urn:btih:..."
magnet_link = input("请输入您的磁力链接地址，并按回车键: ")

# 设置下载参数
params = lt.parse_magnet_uri(magnet_link)
params.save_path = '/content/drive/Shareddrives/shareDrives/download'  # 设置文件保存路径
h = ses.add_torrent(params)
s = h.status()

print(f"开始下载: {s.name}")

try:
    while not h.status().is_seeding:
        s = h.status()
        state_str = [
            '排队中', '检查文件中', '下载元数据中',
            '下载中', '完成', '做种中', '分配空间',
            '检查恢复数据'
        ]

        # 格式化进度输出
        progress_bar = '[' + '#' * int(s.progress * 20) + '-' * (20 - int(s.progress * 20)) + ']'

        print(
            f"\r{state_str[s.state]} {s.progress * 100:.2f}% "
            f"{progress_bar} "
            f"下载速度: {s.download_rate / 1000:.1f} kB/s | "
            f"上传速度: {s.upload_rate / 1000:.1f} kB/s | "
            f"连接节点: {s.num_peers} ",
            end=''
        )

        # 处理会话提醒
        alerts = ses.pop_alerts()
        for alert in alerts:
            if isinstance(alert, lt.metadata_received_alert):
                print("\n元数据接收完成，开始下载文件...")
            # 您可以在此处添加更多提醒处理逻辑
            # pass

        sys.stdout.flush()
        time.sleep(1)


    print("\n\n下载完成，开始做种...")
    # 让其做种一段时间，或者您可以直接按 Ctrl+C 退出
    time.sleep(10)


except KeyboardInterrupt:
    print("\n下载被中断。")

finally:
    # 暂停会话并保存恢复数据
    ses.pause()
    print("\n会话已暂停。")

请输入您的磁力链接地址，并按回车键: magnet:?xt=urn:btih:38DD4FE0CCB32BF7BE4102DB10D181C42B671228&dn=Come+Home+%282023%29+%5B1080p%5D+%5BYTS.MX%5D&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fopen.tracker.cl%3A1337%2Fannounce&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=https%3A%2F%2Fopentracker.i2p.rocks%3A443%2Fannounce
开始下载: Come Home (2023) [1080p] [YTS.MX]
下载中 99.99% [###################-] 下载速度: 866.9 kB/s | 上传速度: 121.5 kB/s | 连接节点: 37 

下载完成，开始做种...

会话已暂停。


## 使用BT种子文件（.torrent）下载

In [ ]:
import libtorrent as lt
import time
import sys
import os
from google.colab import files # 导入colab的files模块


# --- Section 1: 用户上传种子文件 ---
print("请选择一个 .torrent 种子文件进行上传：")
uploaded = files.upload()

if not uploaded:
    print("\n没有上传文件，程序退出。")
    sys.exit()

# 获取上传的第一个文件的文件名
# 假设用户只上传一个种子文件
TORRENT_FILE_PATH = list(uploaded.keys())[0]

# 将内存中的文件内容写入到Colab的磁盘上
with open(TORRENT_FILE_PATH, 'wb') as f:
    f.write(uploaded[TORRENT_FILE_PATH])

print(f"\n'{TORRENT_FILE_PATH}' 已准备就绪，开始下载任务...")
time.sleep(2) # 等待2秒，让用户看清信息



# --- Section 2: libtorrent 下载代码  ---
# 设置文件下载后保存的路径
SAVE_PATH = '/content/drive/Shareddrives/shareDrives/download'

# 检查种子文件是否存在
if not os.path.exists(TORRENT_FILE_PATH):
    print(f"错误: 种子文件未找到 '{TORRENT_FILE_PATH}'")
    sys.exit(1)

# 1. 创建一个 libtorrent 会话
ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})
print("启动 libtorrent 会话...")

# 2. 加载种子文件并设置下载参数
params = {
    'save_path': SAVE_PATH,
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
}
# 从种子文件加载信息
info = lt.torrent_info(TORRENT_FILE_PATH)
params['ti'] = info
h = ses.add_torrent(params)

print(f"开始下载: {info.name()}")
print(f"文件将保存在: {os.path.abspath(SAVE_PATH)}")


# 3. 循环监控下载进度
try:
    while not h.status().is_seeding:
        s = h.status()

        # 状态描述映射
        state_str = [
            '排队中', '检查文件中', '下载元数据中',
            '下载中', '完成', '做种中',
            '分配空间', '检查恢复数据'
        ]

        # 构造进度条
        progress_bar = '[' + '#' * int(s.progress * 20) + ' ' * (20 - int(s.progress * 20)) + ']'

        # 格式化输出字符串
        print(
            f"\r{state_str[s.state]:<12} "
            f"{s.progress * 100:.2f}% "
            f"{progress_bar} "
            f"下载: {s.download_rate / 1000:.1f} kB/s | "
            f"上传: {s.upload_rate / 1000:.1f} kB/s | "
            f"连接数: {s.num_peers} ",
            end=''
        )

        sys.stdout.flush()
        time.sleep(1)

    print("\n\n下载完成，开始做种...")
    # 让其做种一段时间，或者您可以直接按 Ctrl+C 退出
    time.sleep(10)


except KeyboardInterrupt:
    print("\n\n下载被用户中断。")

finally:
    # 4. 暂停会话并保存恢复数据
    ses.pause()
    print("会话已暂停。")